# Faire du calcul parallèle avec R
Reprenons notre code qui utilise les données de différents sites sur plusieurs jours
Supposons que nous voulons tester plusieurs choses à la fois et en faire des graphiques. Il faudrait par exemple créer des boucles qui permettraient de produire automatiquement les graphiques pour chaque modèle.

In [ ]:
specimens <- list(seq(1,50,1))
jours <- 10
nb.times <- length(specimens)*jours

# premiere table
table.specimen <- data.frame(ID=integer(), jour=integer(), hauteur = numeric(), masse = numeric(), BMI=numeric())

for(i in specimens){ # une table par specimen
  table.temp <- data.frame(ID=integer(), jour=integer(), hauteur = numeric(), masse = numeric(), BMI=numeric())
  for(j in 1:jours){ # 10 jours de mesures
    haut <- runif(n = 1, min = 5.4, max = 6.2)
    mass <- sample(98:160, 1)
    BMI <- haut/mass
    temp2 <- data.frame(ID=i, jour=j, hauteur = haut, masse = mass, BMI=BMI)
    table.temp <- rbind(table.temp, temp2)
  }
  table.specimen <- rbind(table.specimen, table.temp)
}

# deuxieme table
table.alloc1 <- as.data.frame(matrix(ncol=5, nrow=length(specimens)*jours)) # nombre de lignes = nb specimens *nb jours
names(table.alloc1) <- c("ID", "jour", "hauteur", "masse", "BMI")
ligne <- 1
for(i in specimens){ # une table par specimen
  for(j in 1:jours){ # 10 jours de mesures
    haut <- runif(n = 1, min = 5.4, max = 6.2)
    mass <- sample(98:160, 1)
    BMI <- haut/mass
    table.alloc1[ligne,] <- c(i,j,haut,mass,BMI)

    ligne <- ligne+1
  }
}

# 3e table
table.alloc2 <- data.frame(ID=rep(specimens, jours), 
                           jour=rep(1:jours, length(specimens)), 
                           hauteur = replicate(1,runif(min= 5.4, max= 6.2,n=nb.times)), 
                           masse = replicate(1,sample(98:160, nb.times, replace = TRUE)), 
                           BMI=numeric(length = nb.times))
table.alloc2$BMI <- table.alloc2$hauteur/table.alloc2$masse

Nous pourrions vouloir imprimer des graphiques ou sortir des coefficients d'un modèle (ex: régression linéaire) en utilisant une boucle.

In [ ]:
for(indiv in specimens){
  df.subset <- subset(table.alloc2, ID == indiv, select = c(names(table.alloc2)))
  plot(df.subset)
}

In [ ]:
for(j in 1:jours){
  df.subset <- subset(table.alloc2, jour == j, select = c(names(table.alloc2)))
  lm.model <- as.numeric(lm(BMI~hauteur, data = df.subset)$coefficients[2])
  print(lm.model)
}

Mais si nous avons une centaine de spécimens que nous voulons analyser séparément, ça pourrait rapidement devenir très long. Dans l'exemple ici il s'agit d'un modèle linéaire, mais si on pensait faire un modèle segmenté avec plusieurs points de convergence, nous pourrions nous retrouver avec quelques secondes/quelques minutes par itération. Si chaque itération prend 1 minute et que nous devons faire 100 itérations, il devient clair que la boucle en série n'est pas le meilleur choix. Nous pouvons à ce moment utiliser doParallel, qui nous permettra de passer une série d'itérations en simultané.

In [ ]:
library(doParallel)

Une étape importante est de transformer le contenu de notre boucle série en fonction, qui pourra être réutilisée dans la boucle foreach.

In [ ]:
print.coeff <- function(indiv){
  df.subset <- subset(table.alloc2, ID == indiv, select = c(names(table.alloc2)))
  lm.model <- as.numeric(lm(BMI~hauteur, data = df.subset)$coefficients[2])
  print(lm.model)
}


Nous pouvons ensuite utiliser cette fonction dans une boucle foreach, qui a une syntaxe un peu différente de ce à quoi certaines personnes sont habituées. Chose à noter: toute la boucle doit se trouver sur une même ligne pour être lue de manière adéquate.

In [ ]:
foreach(indiv=specimens, .combine=rbind) %dopar% {print.coeff(indiv=indiv)}

Ici, nous utilisons la fonction print.coeff dans la partie exécutée de la boucle foreach. Vous noterez l'avertissement que retourne R. Il indique que foreach peut fonctionner en série si aucun nombre de coeurs à utiliser n'a été déterminé. À faire attention lorsqu'on produit le code!

In [ ]:
ncores = 2
registerDoParallel(cores=ncores)# Shows the number of Parallel Workers to be used

In [ ]:
# ici, nous avons la manière de specifier le nombre de coeurs a utiliser lorsque la valeur est dependante du fichier
# de soumission
#ncores = Sys.getenv("SLURM_CPUS_PER_TASK") 

#registerDoParallel(cores=ncores)# Shows the number of Parallel Workers to be used

In [ ]:
foreach(indiv=specimens, .combine=rbind) %dopar% {print.coeff(indiv=indiv)}

Et voilà! Du code parallèle sur deux coeurs! Ça pourrait fonctionner sur votre ordinateur personnel, tout comme sur un supercalculateur!